# Install the package

In [1]:
%pip install git+https://github.com/chenming-pu/cna-inferer.git

  Cloning https://github.com/chenming-pu/cna-inferer.git to c:\users\12039\appdata\local\temp\pip-req-build-uiti_2qv
  Resolved https://github.com/chenming-pu/cna-inferer.git to commit 1ebe21190457e6c08d4a508a8c85d7f857ff0b5a
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for cna-inferer: filename=cna_inferer-0.1.0-py3-none-any.whl size=7800 sha256=a186eb928d50241fe63abfb0065475cae0ed21d25e4e164232061af9267b3830
  Stored in directory: C:\Users\12039\AppData\Local\Temp\pip-ephem-wheel-cache-lke5f9qo\wheels\47\92\dc\b7958e21c265f61f93b71bcf639f4d302b01338d71751f9b87
Successfully built cna-inferer
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/chenming-pu/cna-inferer.git 'C:\Users\12039\AppData\Local\Temp\pip-req-build-uiti_2qv'
  DEPRECATION: Building 'cna-inferer' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'cna-inferer'. Discussion can be found at https://github.com/pypa/pip/issues/6334


# Initialization

In [2]:
# Standard library
import itertools
from collections import defaultdict

# Third-party libraries


# Local package: cna_inferer
from cna_inferer.main import process_and_call_cnas

import urllib.request
import gzip
import shutil
import os


# Download the GFT file

The file will be download to your currnet folder - make sure it is the same one where you place your dataset and the main code (this example code)

In [3]:

url = "https://ftp.ensembl.org/pub/release-104/gtf/homo_sapiens/Homo_sapiens.GRCh38.104.gtf.gz"
gz_file = "Homo_sapiens.GRCh38.104.gtf.gz"
gtf_file = "Homo_sapiens.GRCh38.104.gtf"


if not os.path.exists(gz_file):
    print("Downloading GTF file...")
    urllib.request.urlretrieve(url, gz_file)
    print("Download complete ✅")
else:
    print("File already exists, skipping download.")


if not os.path.exists(gtf_file):
    print("Decompressing...")
    with gzip.open(gz_file, 'rb') as f_in, open(gtf_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    print("Decompression complete ✅")
else:
    print("GTF already extracted.")

Download complete ✅
Decompressing...
Decompression complete ✅


# Load your dataset

Again, make sure your dataset is in the same forlder as this example code

In [5]:
# === Parameters ===
# h5_file:      Path to your input AnnData file (.h5 or .h5ad) with raw scRNA-seq counts
# gtf_file:     Gene annotation file in GTF format (default: Homo_sapiens.GRCh38.104.gtf)
# window_size:  Number of genes per sliding window (larger = smoother signal)
# z_thresh:     Z-score threshold to call gain/loss events
# min_bins:     Minimum number of consecutive bins to consider a valid CNA
# n_bkps:       Max number of breakpoints for segmentation (if model="l2")
# model:        Segmentation model: "l2", "rbf", or other supported by ruptures
# n_jobs:       Number of CPU cores for parallel processing

# === Example Run ===
adata = process_and_call_cnas(
    # h5_file="Your dataset name",                        # Your input dataset- only support h5 or h5ad format
    h5_file= "GSM3814890_day8_rep3_filtered_gene_bc_matrices_h5.h5", # the example dataset
    gtf_file="Homo_sapiens.GRCh38.104.gtf",                 # Downloaded GTF annotation
    window_size=150,                                       # Sliding window size
    z_thresh=1.5,                                          # Gain/loss threshold
    min_bins=3,                                            # Minimum CNA region size
    n_bkps=4,                                              # Max breakpoints for segmentation
    model="l2",                                            # Segmentation model
    n_jobs=1                                               # Use 1 CPU cores
)

# === View output ===
print("Binned region info:")
print(adata.uns["bin_info"].head())

print("\nPer-cell CNA events:")
print(adata.uns["cna_events"].head())

print("\nPer-cell gain/loss assignment:")
print(adata.obs["cna_calls"].value_counts())

c:\Users\12039\anaconda3\Lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'tag', 'ccds_id', 'transcript_support_level', 'exon_number', 'exon_id', 'exon_version', 'protein_id', 'protein_version']
c:\Users\12039\anaconda3\Lib\site-packages\scanpy\preprocessing\_simple.py:287: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["n_cells"] = number


✅ Processed 0 cells...
✅ Processed 500 cells...
✅ Processed 1000 cells...
✅ Processed 1500 cells...
✅ Processed 2000 cells...
✅ Processed 2500 cells...
✅ Processed 3000 cells...
✅ Processed 3500 cells...
✅ Processed 4000 cells...
✅ Processed 4500 cells...
✅ Processed 5000 cells...
✅ Processed 5500 cells...
✅ Processed 6000 cells...
📊 Total CNA events in GSM3814890_day8_rep3_filtered_gene_bc_matrices_h5.h5: 63
Binned region info:
  chromosome  start_gene
0          1     89295.0
1          1   9539465.0
2          1  22025511.0
3          1  32052291.0
4          1  43172330.0

Per-cell CNA events:
                 cell  start_bin  end_bin  type chromosome  start_gene
0  AACGTTGGTTCTGGTA-1        120      124  gain         19  47607873.0
1  ACATACGGTCATACTG-1        120      124  gain         19  47607873.0
2  ACATGGTAGTGGTCCC-1         70       74  gain         11    695591.0
3  ACGCCGACACAGCGTC-1         70       74  gain         11    695591.0
4  ACGCCGACACGCCAGT-1         70       7